In [ ]:
#Add dice coefficient and iou
# use Adadelta as optimizer
# use monitor='val_dice_coefficient' when training 
## use monitor='val_dice_coefficient' when training may have a problem

In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
#metrics
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # 为了避免除以零
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred):
    smooth = 1e-6
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[1,2,3])
    iou = (intersection + smooth) / (sum_ - intersection + smooth)
    return iou


In [2]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np



from tensorflow.keras.optimizers import Adam

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

from tensorflow.keras.optimizers import Adadelta


def dice_loss(y_true, y_pred):
    """
    Dice loss, a measure of overlap between two samples.
    """
    smooth = 1.0  # Smooth factor to avoid division by zero
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + smooth)



def unet_binary(pretrained_weights=None, input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    normalize1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(normalize1)
    #池化后做attention
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    normalize2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(normalize2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    normalize3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(normalize3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    normalize4 = BatchNormalization()(conv4)
    #drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(normalize4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    
    #drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv5))
    normalize6 = BatchNormalization()(up6)
    merge6 = concatenate([normalize4, normalize6], axis=3)
    
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    
    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    normalize7 = BatchNormalization()(up7)
    merge7 = concatenate([normalize3, normalize7], axis=3)
    
    #merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv7))
    normalize8 = BatchNormalization()(up8)
    
    
    merge8 = concatenate([normalize2, normalize8], axis=3)
    
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv8))
    normalize9 = BatchNormalization()(up9)
    
    merge9 = concatenate([normalize1, normalize9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Dropout(0.2)(conv9)#, training=True)
    
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)  # Sigmoid activation for binary output

    model = Model(inputs=inputs, outputs=conv10)
    
    model.compile(optimizer=Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-8), 
              loss=dice_loss, 
              metrics=[dice_coefficient, iou])

    #model.compile(optimizer=Adam(learning_rate=1e-5), loss=dice_loss, metrics=['accuracy'])

    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model



In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from __future__ import print_function
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.callbacks import ReduceLROnPlateau



def preprocess_image(image):
    # 对数变换
    image = np.log1p(image)
    # 归一化到 [0, 1]
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return image

        
def split_train_val(image_folder, mask_folder, val_size):
    image_paths = glob.glob(os.path.join(image_folder, '*.tif'))  # 假设使用png格式，根据实际情况修改
    mask_paths = glob.glob(os.path.join(mask_folder, '*.png'))  # 假设掩模也是png格式

    # 确保图像和掩模是匹配的
    image_paths.sort()
    mask_paths.sort()

    # 划分训练集和验证集
    train_images, val_images, train_masks, val_masks = train_test_split(
        image_paths, mask_paths, test_size=val_size, random_state=42
    )
    
    return train_images, train_masks, val_images, val_masks

def create_datagen(image_paths, mask_paths, batch_size, target_size):
    # 将图像和掩码路径列表转换为 pandas DataFrame
    image_df = pd.DataFrame({'filename': image_paths})
    mask_df = pd.DataFrame({'filename': mask_paths})

    image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
    mask_datagen = ImageDataGenerator()

    # 使用 DataFrame 代替列表
    image_generator = image_datagen.flow_from_dataframe(
        dataframe=image_df,
        x_col='filename',
        y_col=None,
        class_mode=None,
        color_mode='grayscale',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False
    )

    mask_generator = mask_datagen.flow_from_dataframe(
        dataframe=mask_df,
        x_col='filename',
        y_col=None,
        class_mode=None,
        color_mode='grayscale',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False
    )

    while True:
        img = next(image_generator)
        mask = next(mask_generator)
        yield img, mask

# 使用示例
batch_size = 2
target_size = (256, 256)

# 调整为实际图像和掩模的路径
image_folder = 'data/filter/image/band_1'
mask_folder = 'data/filter/labels'

# 现在可以使用train_gen和val_gen进行模型训练和验证了
# 假设 train_images 和 val_images 是通过之前的分割得到的训练和验证图像路径列表
train_images, train_masks, val_images, val_masks = split_train_val(image_folder, mask_folder, val_size=0.2)

# 计算 steps_per_epoch 和 validation_steps
steps_per_epoch = np.ceil(len(train_images) / batch_size)
validation_steps = np.ceil(len(val_images) / batch_size)

# 创建数据生成器
train_gen = create_datagen(train_images, train_masks, batch_size, target_size)
val_gen = create_datagen(val_images, val_masks, batch_size, target_size)



In [4]:
#use Adadelta进行自调节，取消外部回调实例

In [6]:




model = unet_binary()
model_checkpoint = ModelCheckpoint('unet_band1_val_best.hdf5', monitor='val_loss',verbose=1, save_best_only=True)

#model_checkpoint = ModelCheckpoint('best_model.hdf5', monitor='val_dice_coefficient', verbose=1, save_best_only=True)

# 创建 ReduceLROnPlateau 回调实例
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# 模型训练
model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,  # 使用计算得到的值
    epochs=50,
    callbacks=[model_checkpoint],
    validation_data=val_gen,
    validation_steps=validation_steps  # 使用计算得到的值
)








Epoch 1/50
160/160 [==============================] - 68s 419ms/step - loss: 0.5765 - dice_coefficient: 0.4239 - iou: 0.2538 - val_loss: 0.9044 - val_dice_coefficient: 0.0955 - val_iou: 0.0461

Epoch 00001: val_loss improved from inf to 0.90445, saving model to unet_band1_val_best.hdf5
Epoch 2/50
160/160 [==============================] - 67s 418ms/step - loss: 0.5487 - dice_coefficient: 0.4517 - iou: 0.2781 - val_loss: 0.6441 - val_dice_coefficient: 0.3559 - val_iou: 0.2234

Epoch 00002: val_loss improved from 0.90445 to 0.64406, saving model to unet_band1_val_best.hdf5
Epoch 3/50
160/160 [==============================] - 67s 418ms/step - loss: 0.5298 - dice_coefficient: 0.4706 - iou: 0.2937 - val_loss: 0.5567 - val_dice_coefficient: 0.4433 - val_iou: 0.2893

Epoch 00003: val_loss improved from 0.64406 to 0.55668, saving model to unet_band1_val_best.hdf5
Epoch 4/50
160/160 [==============================] - 67s 418ms/step - loss: 0.5099 - dice_coefficient: 0.4905 - iou: 0.3098 - val_

160/160 [==============================] - 67s 421ms/step - loss: 0.2147 - dice_coefficient: 0.7855 - iou: 0.6097 - val_loss: 0.3536 - val_dice_coefficient: 0.6464 - val_iou: 0.5079

Epoch 00031: val_loss did not improve from 0.33485
Epoch 32/50
160/160 [==============================] - 67s 418ms/step - loss: 0.2102 - dice_coefficient: 0.7901 - iou: 0.6156 - val_loss: 0.3714 - val_dice_coefficient: 0.6286 - val_iou: 0.4753

Epoch 00032: val_loss did not improve from 0.33485
Epoch 33/50
160/160 [==============================] - 67s 418ms/step - loss: 0.2042 - dice_coefficient: 0.7960 - iou: 0.6234 - val_loss: 0.3546 - val_dice_coefficient: 0.6454 - val_iou: 0.4990

Epoch 00033: val_loss did not improve from 0.33485
Epoch 34/50
160/160 [==============================] - 67s 419ms/step - loss: 0.1978 - dice_coefficient: 0.8024 - iou: 0.6318 - val_loss: 0.3748 - val_dice_coefficient: 0.6252 - val_iou: 0.4694

Epoch 00034: val_loss did not improve from 0.33485
Epoch 35/50
160/160 [=======